# Fractional Resampling

In this notebook we will implement a simple fractional resampler that uses local Lagrange interpolation. The resampler can be used to perform any rational sampling rate change (but beware of aliasing when downsampling!)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython
from scipy.io import wavfile

In [ ]:
plt.rcParams["figure.figsize"] = (14,4)

Given a nominal input sampling rate $F_i$ and an output sampling rate $F_o$, a fractional resampler generates $A$ output samples for every $B$ input samples where

$$
    \frac{A}{B} = \frac{F_o}{F_i}
$$
and $A, B$ are coprime for maximum efficiency. So the first thing we need is a simple function that simplifies the ratio of sampling frequencies to its lowest terms. For this we will use Euclid's algorithm:

In [ ]:
def simplify(A, B):
    # Euclid's GCD algorithm
    a = A
    b = B
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return A // a, B // b

We can test the function on the usual CD to DVD sampling rate change and indeed we obtain the familiar 160/147 ratio:

In [ ]:
print(simplify(48000, 44100))

A fractional resampler generates each output sample via a local interpolation that uses an odd number of input samples centered around an "anchor" point and a fractional offset $\tau$ from the anchor point where $|\tau| < 1/2$. The resampler pattern repeats every $A$ output point and we need $B$ input point for each output block. 

The method is best understood graphically: a downsampling with a ratio $A/B = 4/5$ generates 4 output samples for every 5 input samples; the anchor points are determined like so:

![title](down.png)

In the figure, the red dotted lines indicate the intervals with $\tau$ less than one half in magnitude. As apparent from the figure, since the output rate is less than the input rate, every once in a while an input sample is skipped.

In the following figure, the sgnal is upsampled with a ratio $A/B = 8/5$ generates 8 output samples for every 5 input samples; since the output rate is larger than the input rate, every once in a while an input sample needs to be reused.

![title](up.png)

Mathematically, the anchor points are determined like so:

 * the output sample $y[m]$ occurs at time $t_m = m/F_o$
 * the closest input sample will be $x[n]$ (occurring at time $t_n = n/F_i$) where $n$ is such that $|m/F_o - n/F_i| < 1/2$.
 
The required value for $n$ can be found by setting 
$$
     \left|\mbox{frac}\left(\frac{m}{A} - \frac{n}{B}\right)\right| < \frac{1}{2}
$$
where frac() indicated the fractional part of a number. This yields
$$
    n = \mbox{round}\left(m\frac{B}{A}\right);
$$
the fractional distance between $y[m]$ and $x[n]$ is given by the time difference normalized by the input's sampling period, that is
$$
    \tau = F_i(m/F_o - n/F_i) = m\frac{B}{A} - \mbox{round}\left(m\frac{B}{A}\right).
$$
Note that $\tau = 0$ every time $m$ is a multiple of $A$, which confirms the repetition pattern every $A$ output samples.

The following function sets up a set of $A$ quadratic interpolation filters and relative anchor points:

In [ ]:
def setup_filters(output_rate, input_rate):
    A, B = simplify(output_rate, input_rate)
    filterbank = [None] * A
    # while output index spans [0, A-1], the input spans [0, B-1]
    for m in range(0, A):
        anchor = int(m * B / A + 0.5) 
        delta = (m * B / A) - anchor
        filterbank[m] = (
            anchor, 
            np.array([
                delta * (delta - 1) / 2, 
                (1 - delta) * (1 + delta), 
                delta * (delta + 1) / 2
            ]))
    return filterbank

We can test with a simple example:

In [ ]:
setup_filters(3, 2)

We are now ready to write the full interpolation function:

In [ ]:
def resample(output_rate, input_rate, x):
    A, B = simplify(output_rate, input_rate)
    filterbank = setup_filters(A, B)
    
    # prepare an array for the output samples
    num_out_samples = (A * len(x)) // B
    y = np.zeros(num_out_samples)
            
    block = 0
    m = 0
    while m < num_out_samples:
        # offset in the input data
        offset = block * B
        # go through the filters
        for fb in filterbank:
            n = offset + fb[0]
            # let's not overshoot
            if n < len(x) - 1:
                y[m] = x[n-1] * fb[1][0] + x[n] * fb[1][1] + x[n+1] * fb[1][2]
            m += 1
        block += 1
        
    return y

We can now test the resampler on a simple sinusoid; we generate the sinusoid at 44.1 KHz and resample at 48KHz; the pitch should not change:

In [ ]:
x = np.cos(2 * np.pi * 440 / 44100 * np.arange(0, 44100))
IPython.display.Audio(x, rate=44100)

In [ ]:
y = resample(12000, 44100, x)
IPython.display.Audio(y, rate=12000)

We can now test the resampler on an audio file; note how aliasing appears when we downsample too much:

In [ ]:
Fi, x = wavfile.read('oao.wav')
IPython.display.Audio(x, rate=Fi)


In [ ]:
y = resample(48000, Fi, x)
IPython.display.Audio(y, rate=48000)

In [ ]:
y = resample(8000, Fi, x)
IPython.display.Audio(y, rate=8000)